# <center> Adherence in a RCT mHealth dataset </center>

### Introduction
Dieses Notebook stellt verschiedene Algorithmen zur Analyse des Adherence Verhaltens von Personen aus einem gegebenen Datensatz zur Verfügung.

### Imports
Damit wir alle benötigten Algorithmen und Funktionen auch aufrufen können, importieren wir zunächst alle Python Dateien, in denen unser Code geschrieben steht.

In [6]:
import sys
sys.path.append('C:/Users/mauri/PycharmProjects/Softwareprojekt/code')
from helper import *
from task3_prediction import *
from task4_statistics import *

### Read in data
Zuerst lesen wir den Datensatz als pandas DataFrame ein. Der Datensatz muss in einem CSV-Format vorliegen. Dafür muss der gesamte Dateipfad zwischen den Hochkommas hinter "dataset =" eingesetzt werden.

In [7]:
dataset = 'C:/Users/mauri/PycharmProjects/Softwareprojekt/data/gefiltert.csv'

# Einlesen des Datensatzes als pandas DataFrame
df = csv_to_dataframe(dataset)

### Group and sort data
Um mit dem Datensatz besser arbeiten zu können müssen wir diesen zunächst etwas sortieren. Dafür gruppieren wir alle user_ids, sodass die Daten eines Nutzers alle hintereinander stehen und nicht quer verteilt sind und sortieren die Daten nach dem Datum von collected_at, sodass sie auch zeitlich richtig sortiert sind. Das Ergebnis wird in der Datei dataset_sorted.csv gespeichert.

In [8]:
# Gruppieren des DataFrame nach user_id und Sortieren nach collected_at
df_sorted = group_and_sort(df)

# Speichern des gruppierten und sortierten DataFrames in einer neuen CSV-Datei
df_sorted.to_csv("C:/Users/mauri/PycharmProjects/Softwareprojekt/data/dataset_sorted.csv", index=False)
df_sorted.head()

,id,user_id,locale,client,collected_at,created_at,updated_at,value_loudness,value_cumberness,value_jawbone,...,collected_at_cumberness,collected_at_jawbone,collected_at_neck,collected_at_tin_day,collected_at_tin_cumber,collected_at_tin_max,collected_at_movement,collected_at_stress,collected_at_emotion,collected_at_diary_q11
0,1104,2107,en,"{""name"":""UNITI 1.0"",""os"":""iOS 14.5.1"",""device""...",2021-05-14T15:48:21Z,2021-05-14T15:48:21Z,2021-05-14T15:48:21Z,35,3,0,...,2021-05-14T15:45:00Z,2021-05-14T15:46:03Z,2021-05-14T15:45:27Z,2021-05-14T15:46:12Z,2021-05-14T15:46:18Z,2021-05-14T15:46:24Z,2021-05-14T15:46:41Z,2021-05-14T15:46:44Z,2021-05-14T15:46:56Z,2021-05-14T15:48:02Z
1,1162,2107,de,"{""os"":""iOS 14.5.1"",""name"":""UNITI 1.0"",""device""...",2021-05-15T17:32:52Z,2021-05-15T17:32:53Z,2021-05-15T17:32:53Z,35,18,0,...,2021-05-15T17:30:57Z,2021-05-15T17:31:06Z,2021-05-15T17:30:49Z,2021-05-15T17:31:45Z,2021-05-15T17:31:48Z,2021-05-15T17:31:49Z,2021-05-15T17:32:09Z,2021-05-15T17:32:12Z,2021-05-15T17:32:16Z,2021-05-15T17:32:44Z
2,1217,2107,de,"{""os"":""iOS 14.5.1"",""name"":""UNITI 1.0"",""device""...",2021-05-16T17:31:08Z,2021-05-16T17:31:08Z,2021-05-16T17:31:08Z,37,31,0,...,2021-05-16T17:30:09Z,2021-05-16T17:30:13Z,2021-05-16T17:30:15Z,2021-05-16T17:30:20Z,2021-05-16T17:30:25Z,2021-05-16T17:30:28Z,2021-05-16T17:30:33Z,2021-05-16T17:30:36Z,2021-05-16T17:30:39Z,2021-05-16T17:31:00Z
3,1298,2107,de,"{""os"":""iOS 14.5.1"",""device"":""iPhone"",""name"":""U...",2021-05-17T19:12:20Z,2021-05-17T19:12:21Z,2021-05-17T19:12:21Z,26,22,0,...,2021-05-17T19:11:30Z,2021-05-17T19:11:34Z,2021-05-17T19:11:37Z,2021-05-17T19:11:43Z,2021-05-17T19:11:46Z,2021-05-17T19:11:49Z,2021-05-17T19:11:52Z,2021-05-17T19:11:56Z,2021-05-17T19:11:57Z,2021-05-17T19:12:11Z
4,1299,2107,de,"{""os"":""iOS 14.5.1"",""device"":""iPhone"",""name"":""U...",2021-05-17T19:13:10Z,2021-05-17T19:13:10Z,2021-05-17T19:13:10Z,22,28,0,...,2021-05-17T19:12:38Z,2021-05-17T19:12:47Z,2021-05-17T19:12:48Z,2021-05-17T19:12:52Z,2021-05-17T19:12:54Z,2021-05-17T19:12:57Z,2021-05-17T19:13:00Z,2021-05-17T19:13:02Z,2021-05-17T19:13:03Z,NaN


### Prediction of a users adherence behaviour
In dieser Task geht es darum die Adherence eines neuen Nutzers für einen bestimmten Tag y in der Zukunft vorherzusagen. Dies geschieht mithilfe der Daten die von ihm schon zu Verfügung stehen und mithilfe des Datensatzes. Zuerst lesen wir die Daten unseres Nutzers ein und geben den Tag an, den wir vorhersagen wollen. Außerdem bekommt unser Datensatz noch das zusätzliche Attribut day, welches den jeweiligen Tag der Nutzereingabe angibt beginnend mit dem ältesten Datum, welches vom Nutzer vorhanden ist.

In [4]:
new_user = 'C:/Users/mauri/PycharmProjects/Softwareprojekt/data/new_user.csv'
day_y = 20

df_newuser = csv_to_dataframe(new_user)

df_prediction = add_day_attribute(df_sorted)

          id  user_id locale  \
0       1104     2107     en   
1       1162     2107     de   
2       1217     2107     de   
3       1298     2107     de   
4       1299     2107     de   
...      ...      ...    ...   
17592  69768    40414     es   
17593  69829    40414     es   
17594  70033    40414     es   
17595  70089    40414     es   
17596  70218    40414     es   

                                                  client  \
0      {"name":"UNITI 1.0","os":"iOS 14.5.1","device"...   
1      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
2      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
3      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
4      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
...                                                  ...   
17592  {"name":"typ","device":"realme RMX2001","os":"...   
17593  {"name":"typ","device":"realme RMX2001","os":"...   
17594  {"name":"typ","device":"realme RMX2001","os":"...   
17595  {"name":

#### Step 1
Im ersten Schritt suchen wir den Datensatz nach den k-ähnlichsten Personen zu unserem neuen Nutzer ab. Dabei suchen wir nach möglichst ähnlichen Lücken (Anzahl und Länge) im Adherence-Verhalten. Außerdem bekommt jeder Nutzer ein neues Attribut "day_y_adherent", welches mit den Werten true oder false belegt werden kann.

In [10]:
df_similarusers = find_similar_users(df_prediction, df_newuser)
df_similarusers = add_day_y_adherent(df_similarusers, day_y)

          id  user_id locale  \
0       1104     2107     en   
1       1162     2107     de   
2       1217     2107     de   
3       1298     2107     de   
4       1299     2107     de   
...      ...      ...    ...   
17592  69768    40414     es   
17593  69829    40414     es   
17594  70033    40414     es   
17595  70089    40414     es   
17596  70218    40414     es   

                                                  client  \
0      {"name":"UNITI 1.0","os":"iOS 14.5.1","device"...   
1      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
2      {"os":"iOS 14.5.1","name":"UNITI 1.0","device"...   
3      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
4      {"os":"iOS 14.5.1","device":"iPhone","name":"U...   
...                                                  ...   
17592  {"name":"typ","device":"realme RMX2001","os":"...   
17593  {"name":"typ","device":"realme RMX2001","os":"...   
17594  {"name":"typ","device":"realme RMX2001","os":"...   
17595  {"name":

#### Step 2
Im zweiten Schritt trainieren wir einen SVM Klassifikator auf unseren ähnlichen Nutzern aus Step 1, der die Nutzer danach klassifiziert ob sie an Tag y adherent sind oder nicht.

In [11]:
svm_classifier = svm_classification(df_similarusers)

NameError: name 'svm_classification' is not defined

#### Step 3
Im dritten Schritt wollen wir für unseren neuen Nutzer mithilfe unseren in Step 2 trainierten Klassifikators festlegen, ob er an Tag y adherent sein wird oder nicht.

In [ ]:
prediction(svm_classifier, df_newuser)

### Statistics
In dieser Task geht es darum die Phasen und die Gruppen aus Task 1 und 2, sowie einige weitere Statistiken zu einem bestimmten Nutzer in einer Übersicht darzustellen.

In [9]:
user_id = 2107
statistics(df_sorted, user_id)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 

C:\Users/mauri/PycharmProjects/Softwareprojekt/code\task1_phases.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_df[column_name] = pd.to_datetime(user_df[column_name])
